In [ ]:
# pip install scikit-image

In [1]:
%config Completer.use_jedi = False

In [2]:
import os
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
import sys

from sklearn.utils import shuffle

In [3]:
from skimage.util import random_noise
import copy

In [4]:
class Dataset():
    def read_dataset(self):
        directory = "Dataset/"
        train_data = []
        test_data = []
        train_label = [] 
        test_label = []

        for i in range(0,10):
            data_dir = directory + str(i) + '/'

            # bipolar one hot lable
            lable = -1 * np.ones((100, 19), dtype=int)     
            lable[:, i] = 1

            images = self.read_image(data_dir)

            X_train, X_test, y_train, y_test = train_test_split(images, lable, test_size=0.20, random_state=42)

            train_data.append(X_train)
            test_data.append(X_test)
            train_label.append(y_train)
            test_label.append(y_test)

        maths = ['add', 'dec', 'div', 'eq', 'mul', 'sub', 'x', 'y', 'z']
        for i, op in enumerate(maths):
            data_dir = directory + op + '/'

            # bipolar one hot lable
            lable = -1 * np.ones((100, 19), dtype=int)     
            lable[:, i+10] = 1

            images = self.read_image(data_dir)

            X_train, X_test, y_train, y_test = train_test_split(images, lable, test_size=0.20, random_state=42)

            train_data.append(X_train)
            test_data.append(X_test)
            train_label.append(y_train)
            test_label.append(y_test)

        train_data = np.concatenate(train_data, axis=0)
        test_data = np.concatenate(test_data, axis=0)
        train_label = np.concatenate(train_label, axis=0)
        test_label = np.concatenate(test_label, axis=0)

        self.train_data = train_data
        self.test_data = test_data
        self.train_label = train_label
        self.test_label = test_label

        X_train, y_train = shuffle(self.train_data, self.train_label, random_state=0)
        self.train_data_shuffle = X_train
        self.train_label_shuffle = y_train
        
        
        X_test, y_test = shuffle(self.test_data, self.test_label, random_state=0)
        self.test_data_shuffle = X_test
        self.test_label_shuffle = y_test
        
    def read_image(self, directory):
        path = os.path.join(directory)
        images = []
        img_size = 100
        for img in os.listdir(path):
            img_array = cv.imread(os.path.join(path,img), cv.IMREAD_GRAYSCALE)
            if img_array is not None:
                new_image = cv.resize(img_array, (img_size, img_size))
                images.append(new_image / 255)
        
        images = np.array(images)    
        flatten_img = images.reshape(100,(img_size*img_size))
        return flatten_img


In [5]:
class SingleContinuousPerceptronNN():
    def __init__(self, alpha = 1e-6):
#         self.weights = np.random.uniform(0, 1e-5, size=(19, 100*100))
#         self.bias = np.random.uniform(0, 1e-5, size=(19,1))

        self.weights = np.zeros((19, 100*100))
        self.bias = np.zeros((19,1))
        self.alpha = alpha
     
    
    def bipolar_sigmoid(self, y_in):
        e = np.exp(-1 * y_in)
        return (1 - e)/ (1 + e)
#         return (2/ 1+ np.exp(-1 * y_in)) - 1
        
        
    def train(self, X, y_, num_classes = 19, max_epoch = 200, tolerance= 0):
        self.num_classes = num_classes
        
        epoch = 0
        flag = True
        while flag:
            print('epoch', epoch)
#             max_weight_change = 0
            iter = 0
            for f_img, label_ in zip(X, y_):
                y = np.zeros(num_classes)   # output neoron
                
                image = f_img.reshape(1, -1)
                label = label_.reshape(self.num_classes, 1)
                label = label.astype(float)
                
                
                y_in = self.bias + self.weights @ (image.T)
                y = self.bipolar_sigmoid(y_in)
                
                # update weights and bias
                delta = self.alpha * (label - y) * (1 - y*y)  # (19,)
                self.bias += delta

#                 max_delta = np.max(delta)
#                 if abs(max_delta) > max_weight_change:
#                     max_weight_change = abs(max_delta)

                deltaW = delta * image
                self.weights += deltaW

                iter += 1
#                     for i in range(num_classes):
#                         self.weights[i] += delta[i] * f_img
                        
#                         max_delta = np.max(delta[i] * f_img)
#                         if abs(max_delta) > max_weight_change:
#                             max_weight_change = abs(max_delta)
                    
#             if max_weight_change > tolerance:
            if epoch < max_epoch:
                epoch += 1
                flag = True
            else:
                flag = False
            
        return self.weights, self.bias
    
    
    def predict(self, test_img):
        y = np.zeros(self.num_classes)   # output neoron
         
        image = test_img.reshape(1, -1)
        y_in = self.bias + self.weights @ (image.T)
        y = self.bipolar_sigmoid(y_in)

        index = np.argmax(y)
        y = -1* np.ones(self.num_classes, dtype=int)
        y[index] = 1
        return y
    
    
    def test(self, X, y_):
        y_pred = []
        y_true = []
        for f_img, label in zip(X, y_):
            y = self.predict(f_img)
            
            class_predict = np.argmax(y)
            y_pred.append(class_predict)
            
            class_true = np.argmax(label)
            y_true.append(class_true)
            
        self.y_true = y_true
        self.y_pred = y_pred
        
        print('accuracy: ',accuracy_score(self.y_true, self.y_pred)* 100)
        print('\n')
        print('precision: ',precision_score(self.y_true, self.y_pred, average=None, zero_division = 0) * 100)
        print('***')
        print('precision: ',precision_score(self.y_true, self.y_pred, average='macro', zero_division = 0) * 100)
        print('\n')
        print('recall: ', recall_score(self.y_true, self.y_pred, average=None) * 100)
        print('***')
        print('recall: ', recall_score(self.y_true, self.y_pred, average='macro') * 100)
        
        return self.y_true, self.y_pred
    
        

In [6]:
dataset = Dataset()
dataset.read_dataset()

In [7]:
dataset.train_data_shuffle.shape

(1520, 10000)

In [8]:
model = SingleContinuousPerceptronNN(alpha= 0.0001)
weights, bias = model.train(dataset.train_data_shuffle, dataset.train_label_shuffle, max_epoch = 200)

epoch 0
epoch 1
epoch 2
epoch 3
epoch 4
epoch 5
epoch 6
epoch 7
epoch 8
epoch 9
epoch 10
epoch 11
epoch 12
epoch 13
epoch 14
epoch 15
epoch 16
epoch 17
epoch 18
epoch 19
epoch 20
epoch 21
epoch 22
epoch 23
epoch 24
epoch 25
epoch 26
epoch 27
epoch 28
epoch 29
epoch 30
epoch 31
epoch 32
epoch 33
epoch 34
epoch 35
epoch 36
epoch 37
epoch 38
epoch 39
epoch 40
epoch 41
epoch 42
epoch 43
epoch 44
epoch 45
epoch 46
epoch 47
epoch 48
epoch 49
epoch 50
epoch 51
epoch 52
epoch 53
epoch 54
epoch 55
epoch 56
epoch 57
epoch 58
epoch 59
epoch 60
epoch 61
epoch 62
epoch 63
epoch 64
epoch 65
epoch 66
epoch 67
epoch 68
epoch 69
epoch 70
epoch 71
epoch 72
epoch 73
epoch 74
epoch 75
epoch 76
epoch 77
epoch 78
epoch 79
epoch 80
epoch 81
epoch 82
epoch 83
epoch 84
epoch 85
epoch 86
epoch 87
epoch 88
epoch 89
epoch 90
epoch 91
epoch 92
epoch 93
epoch 94
epoch 95
epoch 96
epoch 97
epoch 98
epoch 99
epoch 100
epoch 101
epoch 102
epoch 103
epoch 104
epoch 105
epoch 106
epoch 107
epoch 108
epoch 109
epoch 110


In [9]:
# alpha 0.0001
# 200 epoch
print('Test')
test_y_true , test_y_pred = model.test(dataset.test_data_shuffle, dataset.test_label_shuffle)
print(100*'/')
print('\n')
print('Train')
train_y_true , train_y_pred = model.test(dataset.train_data_shuffle, dataset.train_label_shuffle)

Test
accuracy:  40.26315789473684


precision:  [28.57142857 50.         41.17647059 25.         58.33333333 63.63636364
 50.         19.44444444 40.         18.42105263 78.57142857 69.56521739
 75.         47.82608696 54.16666667 30.43478261 41.17647059  8.33333333
 66.66666667]
***
precision:  45.595986630959814


recall:  [10. 55. 35. 25. 35. 35. 20. 35. 30. 35. 55. 80. 30. 55. 65. 70. 35. 10.
 50.]
***
recall:  40.263157894736835
////////////////////////////////////////////////////////////////////////////////////////////////////


Train
accuracy:  73.75


precision:  [97.87234043 68.75       78.66666667 56.25       85.48387097 77.10843373
 90.16393443 61.8556701  82.60869565 51.2195122  98.38709677 82.41758242
 98.24561404 70.58823529 80.48780488 53.48837209 83.58208955 58.41584158
 89.85507246]
***
precision:  77.12878069809038


recall:  [57.5  68.75 73.75 56.25 66.25 80.   68.75 75.   71.25 78.75 76.25 93.75
 70.   75.   82.5  86.25 70.   73.75 77.5 ]
***
recall:  73.75


## Noise

In [10]:
def noise_img(test_data, amount= 0.1):
    new_test = []
    for img in test_data:
        temp_img = copy.deepcopy(img)
        sp = random_noise(temp_img, mode = 's&p', amount = amount)
        new_test.append(sp)
        
    return np.array(new_test)

In [11]:
new_train_10_noise = noise_img(dataset.train_data_shuffle, amount= 0.1)
new_train_10_noise.shape

(1520, 10000)

In [12]:
new_train_20_noise = noise_img(dataset.train_data_shuffle, amount= 0.2)
new_train_20_noise.shape

(1520, 10000)

In [14]:
# alpha 0.0001
# 200 epoch
print('Train with 10% noise')
test_y_true , test_y_pred = model.test(new_train_10_noise, dataset.train_label_shuffle)


Train with 10% noise
accuracy:  72.89473684210527


precision:  [ 97.77777778  63.95348837  77.14285714  53.48837209  86.20689655
  79.48717949  91.22807018  58.25242718  88.88888889  50.79365079
 100.          78.125       98.24561404  73.17073171  82.5
  51.49253731  82.35294118  60.20408163  87.5       ]
***
precision:  76.88476391221371


recall:  [55.   68.75 67.5  57.5  62.5  77.5  65.   75.   70.   80.   76.25 93.75
 70.   75.   82.5  86.25 70.   73.75 78.75]
***
recall:  72.89473684210526


In [15]:
# alpha 0.0001
# 200 epoch
print('Train with 20% noise')
test_y_true , test_y_pred = model.test(new_train_20_noise, dataset.train_label_shuffle)


Train with 20% noise
accuracy:  72.10526315789474


precision:  [ 82.60869565  62.35294118  82.8125      53.01204819  87.5
  78.04878049  88.52459016  59.22330097  88.52459016  50.79365079
 100.          73.26732673  98.24561404  71.42857143  88.
  50.          76.8115942   60.3960396   95.38461538]
***
precision:  76.15446626260109


recall:  [47.5  66.25 66.25 55.   61.25 80.   67.5  76.25 67.5  80.   73.75 92.5
 70.   75.   82.5  88.75 66.25 76.25 77.5 ]
***
recall:  72.10526315789473


In [16]:
new_test_10_noise = noise_img(dataset.test_data_shuffle, amount= 0.1)
new_test_10_noise.shape

(380, 10000)

In [17]:
new_test_20_noise = noise_img(dataset.test_data_shuffle, amount= 0.2)
new_test_20_noise.shape

(380, 10000)

In [18]:
# alpha 0.0001
# 200 epoch
print('Test with 10% noise')
test_y_true , test_y_pred = model.test(new_test_10_noise, dataset.test_label_shuffle)


Test with 10% noise
accuracy:  37.631578947368425


precision:  [28.57142857 47.36842105 35.71428571 15.38461538 50.         60.
 66.66666667 19.51219512 38.46153846 16.66666667 76.92307692 61.53846154
 75.         45.45454545 56.52173913 28.         43.75       12.
 69.23076923]
***
precision:  44.56654789037221


recall:  [10. 45. 25. 20. 30. 30. 20. 40. 25. 30. 50. 80. 30. 50. 65. 70. 35. 15.
 45.]
***
recall:  37.631578947368425


In [19]:
# alpha 0.0001
# 200 epoch
print('Test with 20% noise')
test_y_true , test_y_pred = model.test(new_test_20_noise, dataset.test_label_shuffle)


Test with 20% noise
accuracy:  35.78947368421053


precision:  [33.33333333 45.45454545 46.66666667 14.81481481 30.         46.15384615
 50.         25.58139535 37.5        17.94871795 75.         68.18181818
 75.         56.25       52.17391304 27.11864407 30.76923077  8.69565217
 46.66666667]
***
precision:  41.43732866440343


recall:  [10. 50. 35. 20. 15. 30. 15. 55. 15. 35. 45. 75. 30. 45. 60. 80. 20. 10.
 35.]
***
recall:  35.78947368421053
